In [ ]:
# For some reason, MySQLWorkbench took 40 mins to load a csv file and didn't work in the end.
# Solution, I can do everything in Python 


In [ ]:
# Modules imported
import pandas as pd
import numpy as np
import matplotlib as plt

In [ ]:
data = pd.read_csv('/Users/paxton615/Downloads/UserBehavior.csv')

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data.isnull().sum()

In [ ]:
data.nunique()

In [ ]:
# col names = ['user_id', 'item_id','category_id','status',"timestamp"]
# pv: page view, buy: buy, cart: cart:, fav:favorite
data.columns = ['user_id', 'item_id','category_id','status',"timestamp"]

In [ ]:
fun = data.head()
fun

In [ ]:
str(pd.Timestamp(fun['timestamp'][0], unit='s'))[:10] 

In [ ]:
# str to timestamp
str(pd.Timestamp('2017-11-11').year)+'-'+str(pd.Timestamp('2017-11-11').month)

In [ ]:
pd.Timestamp(fun['timestamp'][0], unit='s').hour

In [ ]:
pd.Timestamp(fun['timestamp'][0], unit='s').month

In [ ]:
# select 3 million for analysis, calling them 'users'
users = data.iloc[5000000:8000000,:]

In [ ]:
# saving partial data as a new csv file safe time for future restart kernel
# This step might be done multiple times!!!

In [ ]:
users.shape

In [ ]:
users.reset_index(drop=True, inplace=True)

In [ ]:
users.head()

In [ ]:
users.nunique()

In [ ]:
total_nuni = ['1 987994','2268318 4162024','2520377 9439','pv 4','1511544070 815859']

In [ ]:
int(total_nuni[0].split(' ')[1])

In [ ]:
denominator = [int(i.split(' ')[1]) for i in total_nuni]
denominator

In [ ]:
numerator = [i for i in users.nunique()]
numerator

In [ ]:
perct = []
for i,j in zip(numerator,denominator):
    perct.append(i/j)

In [ ]:
        # the subset, users, contains 3% of total users,
perct   # 19% of total items, 73% of categories


In [ ]:
users['hour']=[pd.Timestamp(i, unit='s').hour for i in users.timestamp]

In [ ]:
users['year']=[pd.Timestamp(i, unit='s').year for i in users.timestamp]

In [ ]:
users['day']=[pd.Timestamp(i, unit='s').day for i in users.timestamp]

In [ ]:
users['month']=[pd.Timestamp(i, unit='s').month for i in users.timestamp]

In [ ]:
users.year.unique() # data were generated during 2017-11-25 to 2017-12-3
# 2020,1919,2021 need to be found and cleaned later!

In [ ]:
users[(users['year']==1919)| (users['year']==2021)|(users['year']==2020)].count()

In [ ]:
users = users[users['year']==2017]

In [ ]:
users.head()

In [ ]:
np.sort(users.hour.unique()) # hours seem normal

In [ ]:
users.day.unique()

In [ ]:
users.groupby('day').count() # 11.25-12.3
# the rest of days need to be cleaned

In [ ]:
users = users[users['day'].isin([25,26,27,28,29,30,1,2,3])] 

In [ ]:
users.month.unique()
# data happened during 2017.11-2017.12, some months need to be cleaned

In [ ]:
users[(users['month']==10)| (users['month']==4)|(users['month']==9)|(users['month']==5)|(users['month']==8)].count()

In [ ]:
users = users[users['month'].isin([11,12])]

In [ ]:
users.shape

In [ ]:
users.head() 

In [ ]:
users.to_csv(r'/Users/paxton615/Github_Personal/Alibaba_UserBehavior_Analysis/drafts/users_3m.csv')

# It's time to save a csv file as backup

# Later, just begin from here

In [1]:
# Modules imported
import pandas as pd
import numpy as np
import matplotlib as plt

In [2]:
users = pd.read_csv('/Users/paxton615/Github_Personal/Alibaba_UserBehavior_Analysis/drafts/users_3m.csv')

In [3]:
users.head()

,Unnamed: 0,user_id,item_id,category_id,status,timestamp,hour,year,day,month
0,0,309818,4710383,1792277,pv,1511959603,12,2017,29,11
1,1,309818,1421743,4069500,pv,1511959759,12,2017,29,11
2,2,309818,800137,1216617,pv,1511959828,12,2017,29,11
3,3,309818,2493122,1216617,pv,1511959953,12,2017,29,11
4,4,309818,1461532,3102419,pv,1511998449,23,2017,29,11


In [4]:
users.shape

(2962671, 10)

In [5]:
# dropduplicstes: same user,same day, same hour, viewed the same item 
users = users.drop_duplicates(subset=['user_id','item_id','status','hour','day'])

In [6]:
users.shape

(2716044, 10)

In [7]:
# Form user['date'] by using year, month, day
date = []
for i,j,k in zip(users['year'], users['month'], users['day']):
    date.append(str(i)+"-"+str(j)+"-"+str(k))

In [8]:
users['date']=pd.to_datetime(date)

In [11]:
users.head()

,Unnamed: 0,user_id,item_id,category_id,status,timestamp,hour,year,day,month,date,dayofweek
0,0,309818,4710383,1792277,pv,1511959603,12,2017,29,11,2017-11-29,2
1,1,309818,1421743,4069500,pv,1511959759,12,2017,29,11,2017-11-29,2
2,2,309818,800137,1216617,pv,1511959828,12,2017,29,11,2017-11-29,2
3,3,309818,2493122,1216617,pv,1511959953,12,2017,29,11,2017-11-29,2
4,4,309818,1461532,3102419,pv,1511998449,23,2017,29,11,2017-11-29,2


In [10]:
# Add dayofweek to the party, think it can be intuitively useful later
users['dayofweek']=[pd.Timestamp(i, unit='s').dayofweek for i in users.timestamp]

In [12]:
users.dayofweek.unique()

array([2, 3, 4, 5, 6, 0, 1])

In [13]:
# tidy cols and reset_index
users = users[['user_id', 'item_id' , 'category_id', 'status', 'date','dayofweek' ,'hour']]
users.reset_index(drop=True, inplace=True)

In [14]:
users.dtypes
# Could convert hour to datetime as well, however, don't see the need yet
# pd.to_datetime(users['hour'], format='%H')

user_id                 int64
item_id                 int64
category_id             int64
status                 object
date           datetime64[ns]
dayofweek               int64
hour                    int64
dtype: object

# Tried only keep hour as datetime, haven't gotten lucky
import datetime
from datetime import datetime
from datetime import time

hr=[]
for i in users['hour']:
    hr.append(time(i).strftime('%H'))

users['hr']=[time(i).strftime('%H') for i in users['hour']]

users.hr = pd.to_datetime(users.hr, format='%H')

users.dtypes

users['hr']=users.hr.dt.hour

users.head()

users.dtypes

users['hr']= [datetime.strptime(str(i), '%H') for i in users['hr']]



In [15]:
users.head()

,user_id,item_id,category_id,status,date,dayofweek,hour
0,309818,4710383,1792277,pv,2017-11-29,2,12
1,309818,1421743,4069500,pv,2017-11-29,2,12
2,309818,800137,1216617,pv,2017-11-29,2,12
3,309818,2493122,1216617,pv,2017-11-29,2,12
4,309818,1461532,3102419,pv,2017-11-29,2,23


In [31]:
users.groupby(['date','dayofweek'])[['user_id']].agg(['count'])

,,user_id
,,count
date,dayofweek,
2017-11-25,5,292989
2017-11-26,6,291688
2017-11-27,0,275494
2017-11-28,1,273890
2017-11-29,2,279660
2017-11-30,3,287610
2017-12-01,4,307650
2017-12-02,5,381331


In [ ]:
# Adventurous guess: people love to online shop on Fridays the most

In [32]:
# Count each day's users' status 
#users.groupby(['date','dayofweek','status'])[['user_id']].agg(['count'])

users.groupby('hour').count().sort_values(by = 'user_id', ascending=False).head()

users.groupby('hour').count().sort_values(by = 'user_id').head()

users.groupby('status').count().sort_values(by = 'user_id')